In [84]:
from ela_feature_definition import ela_feature_names
import pandas as pd
import random
import numpy as np
from yellowbrick.cluster import SilhouetteVisualizer

In [61]:
def set_random_seed(seed):
    np.random.seed(seed)
    random.seed(seed)
set_random_seed(42)

In [62]:
ela = pd.read_csv('data/aggregated_ela_representation_dropna.csv',index_col=[0,1,2])
ela=ela[set(ela_feature_names).intersection(ela.columns)]

In [ ]:
from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs

from yellowbrick.cluster import KElbowVisualizer
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
ela_scaled=scaler.fit_transform(ela)
model = KMeans()
visualizer = KElbowVisualizer(model, k=(1,25))
visualizer.fit(ela_scaled)     
visualizer.show()

kmeans = KMeans(n_clusters=visualizer.elbow_value_)
ela['kmeans_cluster']=kmeans.fit_predict(ela_scaled)
ela.groupby('kmeans_cluster').count()


In [63]:
import numpy as np

from matplotlib import pyplot as plt
from scipy.cluster.hierarchy import dendrogram
from sklearn.datasets import load_iris
from sklearn.cluster import AgglomerativeClustering


def plot_dendrogram(model, **kwargs):
    # Create linkage matrix and then plot the dendrogram

    # create the counts of samples under each node
    counts = np.zeros(model.children_.shape[0])
    n_samples = len(model.labels_)
    for i, merge in enumerate(model.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count += 1  # leaf node
            else:
                current_count += counts[child_idx - n_samples]
        counts[i] = current_count

    linkage_matrix = np.column_stack(
        [model.children_, model.distances_, counts]
    ).astype(float)

    # Plot the corresponding dendrogram
    dendrogram(linkage_matrix, **kwargs)

In [64]:
ela.shape

(221, 63)

In [92]:
from sklearn.cluster import *
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
from sklearn.cluster import KMeans, AgglomerativeClustering, SpectralClustering
from sklearn.manifold import TSNE
from sklearn.decomposition import *
from sklearn.metrics.cluster import adjusted_rand_score
from sklearn.pipeline import Pipeline
import numpy as np
from sklearn.mixture import GaussianMixture
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.metrics import pairwise_distances
from sklearn import metrics
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
import matplotlib.cm as cm
import numpy as np
from pathlib import Path
clustering_results={}
df=ela
for i in range(2, 100):
    results_for_cluster={}
    cluster_name = f'cluster_{i}'
    labels =  AgglomerativeClustering(n_clusters=i, affinity='cosine', linkage='complete').fit(df).labels_
    results_for_cluster['labels'] = labels
    samples_per_cluster={k[0]:v for k,v in pd.DataFrame(labels).value_counts().to_dict().items()}
    results_for_cluster['samples_per_cluster']=samples_per_cluster
    results_for_cluster['silhouette_score'] = metrics.silhouette_score(df, labels, metric='cosine')
    sample_silhouette_values = metrics.silhouette_samples(df, labels, metric='cosine')
    clustering_results[cluster_name]=results_for_cluster

In [93]:
clustering_results_df=pd.DataFrame(clustering_results).T

In [94]:
clustering_results_df

,labels,samples_per_cluster,silhouette_score
cluster_2,"[1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","{1: 205, 0: 16}",0.962152
cluster_3,"[0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","{0: 205, 1: 14, 2: 2}",0.933072
cluster_4,"[3, 3, 0, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, ...","{3: 199, 0: 14, 1: 6, 2: 2}",0.907603
cluster_5,"[3, 3, 2, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, ...","{3: 199, 2: 12, 1: 6, 0: 2, 4: 2}",0.905568
cluster_6,"[1, 1, 2, 5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","{1: 199, 2: 12, 0: 6, 4: 2, 3: 1, 5: 1}",0.902641
...,...,...,...
cluster_95,"[52, 8, 49, 47, 51, 40, 76, 40, 17, 40, 53, 61...","{16: 52, 10: 10, 17: 9, 12: 8, 1: 6, 4: 6, 11:...",0.45906
cluster_96,"[52, 8, 49, 95, 51, 40, 76, 40, 17, 40, 53, 61...","{16: 52, 10: 10, 17: 9, 12: 8, 4: 6, 1: 6, 5: ...",0.452432
cluster_97,"[52, 8, 49, 95, 51, 40, 76, 40, 17, 40, 53, 61...","{7: 52, 10: 10, 17: 9, 12: 8, 0: 6, 4: 6, 5: 5...",0.446204
cluster_98,"[52, 8, 49, 95, 51, 40, 76, 40, 17, 40, 53, 61...","{7: 52, 10: 10, 17: 9, 12: 8, 4: 6, 5: 5, 6: 5...",0.442648


In [95]:
clustering_results_df.to_csv('clust.csv')

In [82]:
selected_cluster_counts = [59,73]

In [87]:
metric='cosine'
for cluster_count in selected_cluster_counts:
    agglomerative = AgglomerativeClustering(n_clusters=cluster_count,affinity =metric ,linkage='complete')
    labels=agglomerative.fit(ela).labels_
    elements_per_cluster = pd.DataFrame(labels).value_counts()
    print(elements_per_cluster)
    clusters_with_2_or_less_elements=[x[0] for x in elements_per_cluster[elements_per_cluster<=2].index]
    print(clusters_with_2_or_less_elements)


    ela_copy=ela.copy()
    ela_copy[f'clustering_{cluster_count}_clusters']=labels
    ela_copy.to_csv(f'data/hierarchical_clustering_{cluster_count}.csv')

28    81
6     13
22    13
13    12
42     9
0      7
3      6
5      6
7      6
1      5
4      3
20     3
16     2
46     2
25     2
58     2
10     2
14     2
12     2
2      2
11     2
9      2
50     1
8      1
43     1
44     1
45     1
47     1
48     1
49     1
15     1
40     1
51     1
52     1
53     1
54     1
55     1
56     1
57     1
41     1
37     1
39     1
38     1
18     1
19     1
21     1
23     1
24     1
26     1
27     1
30     1
31     1
32     1
33     1
34     1
35     1
36     1
17     1
29     1
dtype: int64
[16, 46, 25, 58, 10, 14, 12, 2, 11, 9, 50, 8, 43, 44, 45, 47, 48, 49, 15, 40, 51, 52, 53, 54, 55, 56, 57, 41, 37, 39, 38, 18, 19, 21, 23, 24, 26, 27, 30, 31, 32, 33, 34, 35, 36, 17, 29]


AttributeError: 'AgglomerativeClustering' object has no attribute 'predict'